In [1]:
import pandas as pd
import numpy as np

In [15]:
import geocoder

In [2]:
raw = pd.read_table('NormTemperature_ISO_12831.txt')

In [3]:
compiled = pd.DataFrame()

In [4]:
compiled['Avg T'] = raw['Raw'].str.split(' ').str[-1].replace('–','-')

In [5]:
compiled['Min T'] = raw['Raw'].str.split(' ').str[-2].apply(lambda x: x.replace('–','-'))

In [6]:
compiled['Climate Zone'] = raw['Raw'].str.split(' ').str[-3]

In [7]:
compiled['PLZ'] = raw['Raw'].str.split(' ').str[-4]
compiled['PLZ'] = compiled['PLZ'].apply(lambda x: x.split('*')[0])

In [11]:
compiled['Place'] = raw['Raw'].str.split(' ').str[:-4].apply(lambda x: " ".join(x))

In [12]:
compiled.to_csv('T_processed.csv', sep=';')

In [13]:
reprod = pd.read_table('T_processed.csv', sep=';',index_col = 0, decimal = ',', encoding = "ISO-8859-1")

Get longitude and latitude

In [18]:
compiled['LatLng'] = reprod['PLZ'].apply(lambda x: geocoder.google(str(x) + ', Germany').latlng)

In [26]:
compiled['Lat'] = compiled['LatLng'].apply(lambda x: x[0])
compiled['Lng'] = compiled['LatLng'].apply(lambda x: x[1])

In [39]:
geocoder.google(str(compiled[compiled['Lng'] <= 0].loc[16,'PLZ']) + ', Germany').latlng

[51.7793089, 11.4729835]

In [43]:
geocoder.google('Bitterfeld').latlng

[51.6265334, 12.3133061]

In [55]:
for ix in compiled[compiled['Lng'] <= 0].index:
    gg = geocoder.google(str(compiled.loc[ix,'PLZ']) + ', Germany').latlng
    compiled.loc[ix,'Lat'] = gg[0]
    compiled.loc[ix,'Lng'] = gg[1]

In [55]:
for ix in compiled[compiled['Lng'] <= 0].index:
    gg = geocoder.google(str(compiled.loc[ix,'Place']) + ', Germany').latlng
    compiled.loc[ix,'Lat'] = gg[0]
    compiled.loc[ix,'Lng'] = gg[1]

In [56]:
compiled[compiled['Lng'] <= 0]

,Avg T,Min T,Climate Zone,PLZ,Place,LatLng,Lat,Lng


In [58]:
compiled = compiled.drop('LatLng',1)

In [59]:
compiled.to_csv('T_zones_Germany.csv')